In [1]:
from deepface import DeepFace
import face_recognition
from PIL import Image
#from multiprocessing import Pool #mission impossible currently

In [24]:
import torch
import torchvision.transforms as transforms
import numpy as np
from numpy import asarray
import torch.nn.functional as F
from emotion_detection.fer_data_utils import SkResize, HistEq, AddChannel, ToRGB
from vision_utils.custom_architectures import SepConvModel
#extra added
from vision_utils.custom_architectures import SepConvModel, initialize_model, PretrainedMT
from multitask_rag.evaluate import predict_utk as test_image

In [3]:
PATHSEPCONV='/home/manish//Documents/democlassi/emotion-detection/sepconv_model_55_val_loss=1.175765.pth'
PATHRESNETAGR='/home/manish/Documents/democlassi/other/resnet_model_21_val_loss=4.275671.pth'

In [4]:
#emotion
model = SepConvModel()
model.load_state_dict(torch.load(PATHSEPCONV, map_location="cpu"))

#age-race-gender
resnet_model_agr = PretrainedMT(model_name='resnet')
resnet_model_agr.load_state_dict(torch.load(PATHRESNETAGR, map_location="cpu"))

<All keys matched successfully>

In [5]:
data_transforms = transforms.Compose([
    #transforms.Resize((500,500),Image.Resampling.LANCZOS),
    transforms.Resize((200,200),Image.Resampling.LANCZOS),
    transforms.ToTensor()
])

In [ ]:
#these two function are needed doesn't work by just importing these function

In [15]:
def democlassiEvaluate(pil_image,):
    
    img_tensor= data_transforms(pil_image) 
    imge = pil_image.convert('L')
    img = imge.resize((48,48),Image.Resampling.LANCZOS)
    numpydata = asarray(img)
    res1=predict_fer(numpydata,model,True)
    # with resent model--age--race--gender
    #res2=test_image(img_tensor,resnet_model)
    #with sep_conv_model --age--gender---race
    #res2=test_image(img_tensor,sep_conv_model)
    #with VGG model ---age--race--gender
    #res2=test_image(img_tensor,vgg_model_agr)
    
    #for resnet model
    res2 = test_image(img_tensor,resnet_model_agr)
    
    #for sepconv_ model
    #res2 = test_image(img_tensor,sepconv_model_agr)
    return res1,res2 

In [16]:
def deepfaceOutput(obj1):
    print("Deepface comparision")
    print("----Deepface---")
    print("Age:---",{obj1['age']},"---")
    print("Gender:---",{obj1['gender']},"---")
    print("Race:---",{obj1['dominant_race']},"---")
    print("Emotion:---",{obj1['dominant_emotion']},"---")
    print('------------------------------------------- ')


In [21]:
def predict_fer(image, model, transf_learn=True):

    # process image
    image = preprocess_fer(image, transf_learn)
    
    # prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    model = model.to(device)
    image = image.to(device)

    # predict probabilities
    emotion = F.softmax(model(image), dim=1).detach().to('cpu').numpy()[0]
    target_names = ['Angry', 'Disgusted', 'Afraid', 'Happy', 'Sad', 'Surprised', 'Neutral']
    pred_label = target_names[np.argmax(emotion)]

    emotion_probs = dict(zip(target_names, emotion))

    return emotion_probs, pred_label

In [22]:
def preprocess_fer(image, transf_learn):
    if transf_learn:
        transf = transforms.Compose([
            transforms.ToTensor()
        ])
    else:
        transf = transforms.Compose([
            HistEq(),
            AddChannel(),
            transforms.ToTensor()
        ])
        
        
        print(transf(image))  

    return transf(image).to(torch.float32).unsqueeze_(0)

In [17]:
def printDemoClassiOutput(res1,res2):
    print("Democlassi comparision")
    print("---Democlassi---")
    print("Age:---",{round(res2[0], 0)},"---")
    print("Gender:---",{res2[2]},"---")
    print("Race:---",{res2[4]},"---")
    print("Emotion:---",{res1[1]},"---") #res1 for emotion

In [ ]:
import os
from os import listdir
import gc 


folder_dir = "/home/manish/fotos-form-download"
save_dir= "/home/manish/Documents"
for images in os.listdir(folder_dir):
    gc.collect()
    if (images.endswith(".png") or images.endswith(".jpg")or images.endswith(".jpeg")):
            new=folder_dir+"/"+images
            print(new)
            image = face_recognition.load_image_file(new)
            face_locations = face_recognition.face_locations(image, number_of_times_to_upsample=0, model="resnet")
            print("num of Face_locations found: " + str(len(face_locations)))
            
            if len(face_locations) != 0 : #check if the image contains people
                for face_location in face_locations:
                    
                    print("Face location has been found")
                    top, right, bottom, left = face_location
                    face_image = image[top:bottom,left:right]
                    pil_img = Image.fromarray(face_image)
                    pil_image = pil_img.resize((200,200),Image.Resampling.LANCZOS)

                    fileLocation=save_dir+'/'+'tmp.jpg'
                    pil_image.save(fileLocation,"JPEG")
                    
                    print("Deep-face evaluation: \n")
                    obj1= DeepFace.analyze(img_path=fileLocation, enforce_detection=False)
                    deepfaceOutput(obj1)
                    print("Deepface analysis compelete")
                    print("******************************************************************\n")
                    print("democlassi-evaluation:\n")
                    res1, res2 = democlassiEvaluate(pil_image)
                    printDemoClassiOutput(res1,res2)
                    print('------------------------------------------- ')
            else:
                print("Face lOcation not found")
                
                gc.collect()
    print('++++++++++++++++++++++++++++++++++++++++++')

/home/manish/fotos-form-download/image327.jpg
num of Face_locations found: 1
Face location has been found
Deep-face evaluation: 



Action: emotion:   0%|          | 0/4 [00:00<?, ?it/s]

1/1 [==============================] - 0s 45ms/step


Action: age:  25%|██▌       | 1/4 [00:00<00:00,  5.02it/s]    

1/1 [==============================] - 1s 1s/step


...

Action: emotion:   0%|          | 0/4 [00:00<?, ?it/s]

1/1 [==============================] - 0s 24ms/step


Action: age:  25%|██▌       | 1/4 [00:00<00:00,  8.11it/s]    

1/1 [==============================] - 1s 735ms/step


Action: gender:  50%|█████     | 2/4 [00:00<00:01,  1.80it/s]

In [ ]:
#checking if face_recognition can locate all the image position 